<a href="https://colab.research.google.com/github/Puvadol19/BOT-mao/blob/main/%E0%B8%BABOT_MAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 4.1 MB 35.5 MB/s 
     |████████████████████████████████| 1.2 MB 33.9 MB/s 
     |████████████████████████████████| 288 kB 9.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.3 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=3f401a9a3f82cce833c2640997d24146967985e76982ad90511889d9d1425047
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
pip install line_notify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for line-notify: filename=line_notify-0.1.4-py3-none-any.whl size=2655 sha256=8d043e76644c09a50a4be2b0763f2cb54c6cceb750c9cf0747480284b6059ecb
  Stored in directory: /root/.cache/pip/wheels/d6/78/8f/c681e51d54245b3f118c0d3dca03fcbc377ac9e10c6034a2e5
Successfully built line-notify


In [ ]:
pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 3.2 MB/s 


In [ ]:
pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import logging
import ccxt
import pandas as pd
import pandas_ta as ta
import time
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from line_notify import LineNotify
from configparser import ConfigParser
import mplfinance as mplf
import schedule

version = '1.0.0'

logging.basicConfig(filename='log.log', format='%(asctime)s - %(message)s', level=logging.INFO)


dbconf = ConfigParser()
dbconf.read_file(open('/content/config.ini'))
API_KEY = dbconf.get('Config','API_KEY')
API_SECRET = dbconf.get('Config','API_SECRET')
LINE_TOKEN = dbconf.get('Config','LINE_TOKEN')
#เปิด on จะจำกัดจำนวนวนเหรียญ
limitasset = dbconf.get('Limit','limitasset')
nolimitasset = dbconf.get('Limit','nolimitasset')
#เปิด on ใส่เหรียญที่จะให้มองเป็นอันดับแรกๆ
CoinSelectTrade = dbconf.get('Coin_Trade','CoinSelectTrade')
SYMBOL_NAME = dbconf.get('Coin_Trade','SYMBOL_NAME').split(",")
#เปิด on ใส่เหรียญที่ไม่เล่น
Blacklistmode = dbconf.get('Coin_Blacklist','Blacklistmode')
Blacklist = dbconf.get('Coin_Blacklist','Blacklist').split(",")
#เปิด on จะเข้าซื้อขาย
alway= dbconf.get('Mode','Alway_scan')
mode_trade = dbconf.get('Mode','mode_trade')
MarginType = dbconf.get('Mode','MarginType')
TF = dbconf.get('Mode','TF')
#config trade
automaxLeverage = dbconf.get('Trade','automaxLeverage')
cost = dbconf.get('Trade','cost')
Long = dbconf.get('Trade','Long')
Short = dbconf.get('Trade','Short')
NotTrade = dbconf.get('Trade','NotTrade')
#config TPSL
TPSL = dbconf.get('TPSL','TPSLmode')
tp = dbconf.get('TPSL','tp')
sl = dbconf.get('TPSL','sl')
#tailingstop
callback = dbconf.get('TrailingStop','callback')
active = dbconf.get('TrailingStop','active')
#config indicator
EMAposition  = dbconf.get('ParaBot','EMAposition')
EMAfast  = dbconf.get('ParaBot','EMAfast')
EMAslow  = dbconf.get('ParaBot','EMAslow')

#-------------------------------------------------------------------------------

notify = LineNotify(LINE_TOKEN)
print(f'Bot start waited time {str(TF)} for ScAn')
if mode_trade == 'off':
  welcome = f'ยินดีต้อนรับเข้าสู่\nScan Bot MAO MAO ทุกเหรียญ \nversoin:{version}\n\nType : {MarginType}\nBlack list :\n{Blacklist}'
  notify.send(welcome)
else:
  welcome = f'ยินดีต้อนรับเข้าสู่\nScan Bot MAO MAO ทุกเหรียญ \nversoin:{version}\n\nType : {MarginType}\nBlack list :\n{Blacklist}\nLong : {Long}\nShort : {Short}\nTPSL mode :{TPSL}'
  notify.send(welcome)

# API CONNECT
exchange = ccxt.binance({
"apiKey": API_KEY,
"secret": API_SECRET,

'options': {
'defaultType': 'future'
},
'enableRateLimit': True
})

Sside = 'BOTH'
Lside = 'BOTH'
messmode = ''

try:
  currentMODE = exchange.fapiPrivate_get_positionside_dual()
except:
  time.sleep(1)
  currentMODE = exchange.fapiPrivate_get_positionside_dual()

if currentMODE['dualSidePosition']:
  Sside = 'SHORT'
  Lside = 'LONG'
  messmode = 'Hedge Mode'
else:
  messmode = 'One-way Mode'
#-------------------------------------------------------------------------------
def get_symbol():
  symbols = pd.DataFrame()
  syms = []
  print(f'Bot check all coin for trade {TF} {messmode} version = {version}...')
  market = exchange.fetchTickers(params={'type':'future'})
  if len(SYMBOL_NAME) > 0 and CoinSelectTrade == 'on':
    for i in SYMBOL_NAME:
      symbo = i +'/'+MarginType
      syms.append(symbo)
  for x,y in market.items() :
    if y['symbol'][len(y['symbol'])-4:len(y['symbol'])] == MarginType:
      symbols = symbols.append(y , ignore_index=True)
  symbols = symbols.set_index('symbol')
  symbols['datetime'] = pd.to_datetime(symbols['timestamp'], unit='ms', utc=True).map(lambda x: x.tz_convert('Asia/Bangkok'))
  symbols = symbols.sort_values(by=['quoteVolume'],ascending=False)
  if limitasset == 'on':
    symbols = symbols.head(int(nolimitasset))
  newsym = []
  if len(syms) > 0:
    for symbol in syms:
      newsym.append(symbol)
  for symbol in symbols.index:
    newsym.append(symbol)
  if len(Blacklist)>0 and Blacklistmode == 'on':
    for symbol in Blacklist:
      symbo = symbol+'/'+MarginType
      try:
        newsym.remove(symbo)
      except:
        continue
  newsym = list(dict.fromkeys(newsym))
  return newsym
#-------------------------------------------------------------------------------
def indicator(symbol,df):
  try:
    df['emaposition'] = ta.ema(df['Close'],int(EMAposition))
    df['emafast'] = ta.ema(df['Close'],int(EMAfast))
    df['emaslow'] = ta.ema(df['Close'],int(EMAslow))
    data = ta.macd(df['Close'], 12, 26, 9)
    df['MACD'] = data['MACD_12_26_9']
    df['MACDs'] = data['MACDs_12_26_9']    
  except:
    pass
    print(symbol)
    print('Indicator Error NaN')
  return df 
#-------------------------------------------------------------------------------
def feed(symbol,TF):
  bars = exchange.fetch_ohlcv(symbol, timeframe=TF, since = None, limit = 1000)
  df = pd.DataFrame(bars[:-1], columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True).map(lambda x: x.tz_convert('Asia/Bangkok'))
  #df = df.set_index('timestamp')
  df = indicator(symbol,df)
  return df      
#-------------------------------------------------------------------------------
def longEnter(symbol,amount):
  if currentMODE['dualSidePosition']:
    order = exchange.create_market_buy_order(symbol, amount,params={'positionSide':Lside})
  else:
    order = exchange.create_market_buy_order(symbol, amount)
  print("Status : LONG ENTERING PROCESSING...")
  logging.info(order)
  return
#-------------------------------------------------------------------------------
def longClose(symbol,Amt):
  if currentMODE['dualSidePosition']:
    order = exchange.create_market_sell_order(symbol,Amt,params={'positionSide':Lside})
  else:
    order = exchange.create_market_sell_order(symbol,Amt,params={"reduceOnly": True})
  logging.info(order)
  return
#-------------------------------------------------------------------------------
def shortEnter(symbol,amount):
  if currentMODE['dualSidePosition']:
    order = exchange.create_market_sell_order(symbol, amount,params={'positionSide':Sside})
  else:
    order = exchange.create_market_sell_order(symbol, amount)
  print("Status : SHORT ENTERING PROCESSING...")
  logging.info(order)
  return
#-------------------------------------------------------------------------------
def shortClose(symbol,Amt):
  if currentMODE['dualSidePosition']:
    order = exchange.create_market_buy_order(symbol,Amt*-1,params={'positionSide':Sside})
  else:
    order = exchange.create_market_buy_order(symbol,Amt*-1,params={"reduceOnly": True})
  logging.info(order)
  return
#-------------------------------------------------------------------------------
def cancelorder(symbol):
  if currentMODE['dualSidePosition']:
    order = exchange.cancel_all_orders(symbol,params={'positionSide':Lside,'conditionalOrdersOnly': False}) 
  else:
    order = exchange.cancel_all_orders(symbol,params={'conditionalOrdersOnly': False})
  logging.info(order)
  return 
#-------------------------------------------------------------------------------
def longTPSL(symbol,amount,PriceEntry,pricetp,pricesl,pricetpTL):
  if currentMODE['dualSidePosition']:
    order = exchange.create_order(symbol,'TAKE_PROFIT_MARKET','sell',amount,PriceEntry,{'stopPrice':pricetp ,'positionSide':Lside })
    time.sleep(1)
    order = exchange.create_order(symbol,'STOP_MARKET','sell',amount,PriceEntry,{'stopPrice':pricesl,'positionSide':Lside})
    time.sleep(1)
    order = exchange.create_order(symbol,'TRAILING_STOP_MARKET','sell',amount,None,{'activationPrice':pricetpTL,'callbackRate': callback,'positionSide':Lside})
    time.sleep(1)
  else:
    order = exchange.create_order(symbol,'TAKE_PROFIT_MARKET','sell',amount,PriceEntry,{'stopPrice':pricetp ,"reduceOnly": True})
    time.sleep(1)
    order = exchange.create_order(symbol,'STOP_MARKET','sell',amount,PriceEntry,{'stopPrice':pricesl,"reduceOnly": True})
    time.sleep(1)
    order = exchange.create_order(symbol,'TRAILING_STOP_MARKET','sell',amount,None,{'activationPrice':pricetpTL,'callbackRate': callback,"reduceOnly": True})
    time.sleep(1)
  return
#-------------------------------------------------------------------------------
def shortTPSL(symbol,amount,PriceEntry,pricetp,pricesl,pricetpTL):
  if currentMODE['dualSidePosition']:
    order = exchange.create_order(symbol,'TAKE_PROFIT_MARKET','buy',amount,PriceEntry, {'quantityIsRequired' : False,'stopPrice': pricetp ,'positionSide':Sside})
    time.sleep(1)
    order = exchange.create_order(symbol,'STOP_MARKET','buy',amount,PriceEntry, {'quantityIsRequired' : False,'stopPrice': pricesl,'positionSide':Sside})                   
    time.sleep(1)
    order = exchange.create_order(symbol,'TRAILING_STOP_MARKET','buy',amount,None,{'quantityIsRequired' : False,'activationPrice':pricetpTL,'callbackRate': callback,'positionSide':Sside})
    time.sleep(1)
  else:
    order = exchange.create_order(symbol,'TAKE_PROFIT_MARKET','buy',amount,PriceEntry, {'quantityIsRequired' : False,'stopPrice': pricetp ,"reduceOnly": True})
    time.sleep(1)
    order = exchange.create_order(symbol,'STOP_MARKET','buy',amount,PriceEntry, {'quantityIsRequired' : False,'stopPrice': pricesl,"reduceOnly": True})                   
    time.sleep(1)
    order = exchange.create_order(symbol,'TRAILING_STOP_MARKET','buy',amount,None,{'quantityIsRequired' : False,'activationPrice':pricetpTL,'callbackRate': callback,"reduceOnly": True})
    time.sleep(1)
  return
#-------------------------------------------------------------------------------
def Line(symbol,df,LV1,maximum_price,minimum_price):
    df = df.set_index('timestamp')
    df['LV1']=LV1
    df['maximum_price']=maximum_price
    df['minimum_price']=minimum_price

    data = df.tail(100)
    dataLV1 = df['LV1'].tail(100)
    datamaximum_price = df['maximum_price'].tail(100)
    dataminimum_price = df['minimum_price'].tail(100)

    emaposition = df['emaposition'].tail(100)

    MACD = df["MACD"].tail(100)
    MACDs = df["MACDs"].tail(100)
    colors = ['g' if h >= 0 else 'r' for h in MACD]
    MACDplot = mplf.make_addplot(
        MACD,
        type='bar',
        color=colors,
        panel=2,
        ylabel="MACD",
    )
    

    dataLV1plot = mplf.make_addplot(
        dataLV1,linestyle="dashdot", width= 1, color="black",panel=0
    )
    datamaximum_priceplot = mplf.make_addplot(
        datamaximum_price,linestyle="dashdot", width= 0.5, color="red", panel=0
    )
    dataminimum_priceplot = mplf.make_addplot(
        dataminimum_price,linestyle="dashdot", width= 0.5, color="blue", panel=0
    )
    

    emapositionplot = mplf.make_addplot(
        emaposition, color="red", panel=0
    )
    MACDsplot = mplf.make_addplot(
        MACDs, color="red", panel=2
    )



    mplf.plot(
        data,
        volume=True,
        figratio=(8, 6),
        type="candle",
        title=symbol,
        addplot=[
            emapositionplot,
            MACDplot,
            MACDsplot,
            dataLV1plot,
            datamaximum_priceplot,
            dataminimum_priceplot
        ],
        tight_layout=True,
        style="yahoo",
        savefig="order.png",
    )
    notify.send(symbol, image_path="./order.png")
    return
#-------------------------------------------------------------------------------
def trade(symbol,df):
    longposition = False
    shortposition = False
    inposition = False
    Amt =0
    newsymbol = symbol.replace('/','')
    balance = exchange.fetch_balance()
    free_balance = exchange.fetch_free_balance()
    positions = balance['info']['positions']
    current_positions = [position for position in positions if float(position['positionAmt']) != 0 and position['symbol'] == newsymbol]
    position_info = pd.DataFrame(current_positions, columns=["symbol", "entryPrice", "unrealizedProfit", "isolatedWallet", "positionAmt", "positionSide","initialMargin"])
    market_info = pd.DataFrame(exchange.fapiPrivate_get_positionrisk(), columns=["symbol", "entryPrice", "leverage" ,"unrealizedProfit", "isolatedWallet", "positionAmt"])
    if automaxLeverage =='on':
        try:
            market_lv = pd.DataFrame(exchange.fetchMarketLeverageTiers(symbol), columns=["maxLeverage"])
            leverage=int(market_lv["maxLeverage"][0])
            exchange.set_leverage(leverage,newsymbol)
        except:
            leverage = market_info["leverage"][len(market_info.index) - 1]
            exchange.set_leverage(leverage,newsymbol)
    else:
        leverage = market_info["leverage"][len(market_info.index) - 1]

    PriceEntry = float(df["Close"][len(df.index) - 1])
    BalanceEntry = free_balance[MarginType]
    if cost[0]=='#':
        amount = float(cost[1:len(cost)])/PriceEntry
    elif cost[0]=='$' :
        amount = float(cost[1:len(cost)])*float(leverage)/PriceEntry
    else :
        amount =((float(BalanceEntry)/100 ) * float(cost) * float(leverage))/PriceEntry

    if not position_info.empty and position_info["positionAmt"][len(position_info.index) - 1] != 0:
        inposition = True
        Amt = float(position_info["positionAmt"][len(position_info.index) - 1])
    else:
        inposition = False
        shortposition = False
        longposition = False
        cancelorder(symbol)
    if inposition == True and float(position_info["positionAmt"][len(position_info.index) - 1]) > 0:
        longposition = True
        shortposition = False
    if inposition == True and float(position_info["positionAmt"][len(position_info.index) - 1]) < 0:
        shortposition = True
        longposition = False
    try:
        if longposition == True and ( df['emafast'].iloc[-2]>df['emaslow'].iloc[-2] and df['emafast'].iloc[-1]<=df['emaslow'].iloc[-1]):
            longClose(symbol,Amt)
            print("Status : LONG EXIT PROCESSING...")
            notify.send(f'{symbol}\n สถานะ : Long Exit')
            cancelorder(symbol)
        if shortposition == True and ( df['emafast'].iloc[-2]<df['emaslow'].iloc[-2] and df['emafast'].iloc[-1]>=df['emaslow'].iloc[-1]):
            shortClose(symbol,Amt)
            print("Status : SHORT EXIT PROCESSING...")
            notify.send(f'{symbol}\n สถานะ : Short Exit')
            cancelorder(symbol)
        if longposition == False and Long == 'on' and ( df['Close'].iloc[-2]>df['emaposition'].iloc[-2] and df['Close'].iloc[-2]>df['High'].iloc[-2] and df['MACDs'].iloc[-2]>=0):
            if BalanceEntry >= float(NotTrade):
                longEnter(symbol,amount)
                cancelorder(symbol)
                notify.send(f'{symbol}\n สถานะ : Long\nCross Up')
                Line(symbol,df )
                if TPSL =='on':
                    pricetp = PriceEntry +(PriceEntry *(float(tp)/100))
                    pricesl = PriceEntry -(PriceEntry *(float(sl)/100))
                    pricetpTL = PriceEntry +(PriceEntry *(float(active)/100))
                    longTPSL(symbol,amount,PriceEntry,pricetp,pricesl,pricetpTL)
                    print('Set TPSL')
                    notify.send(f'{symbol}\n สถานะ : Long set TPSL\nTP: {tp}%\nSL: {sl}%\nTrailingStop\nCall Back: {callback}%\nActive Price: {round(pricetpTL,5)} {MarginType}')
            else:
                print('over limit cost not order')
        if shortposition == False and Short == 'on' and ( df['Close'].iloc[-2]<df['emaposition'].iloc[-2] and df['Close'].iloc[-2]<df['Low'].iloc[-2] and df['MACDs'].iloc[-2]<=0):
            if BalanceEntry >= float(NotTrade):
                shortEnter(symbol,amount)
                cancelorder(symbol)
                notify.send(f'{symbol}\n สถานะ : Short\nCross Down')
                Line(symbol,df )
                if TPSL =='on':
                    pricetp = PriceEntry -(PriceEntry *(float(tp)/100))
                    pricesl = PriceEntry +(PriceEntry *(float(sl)/100))
                    pricetpTL = PriceEntry -(PriceEntry *(float(active)/100))
                    shortTPSL(symbol,amount,PriceEntry,pricetp,pricesl,pricetpTL)
                    print('Set TPSL')
                    notify.send(f'{symbol}\n สถานะ : Short set TPSL\nTP: {tp}%\nSL: {sl}%\nTrailingStop\nCall Back: {callback}%\nActive Price: {round(pricetpTL,5)} {MarginType}')
            else:
                print('over limit cost not order')
    except ccxt.BaseError as Error:
      pass
      #notify.send(f'\n เกิดปัญหา SCAN Bot{version} : {symbol}\n{Error}')
    return

#-------------------------------------------------------------------------------
def alertonly(symbol,df):
  try:
    TTF = 50
    if TF == '1d':
      TTF = 120
    if TF == '4h':
      TTF = 60
    if TF == '1h':
      TTF = 50
    alert = pd.DataFrame()
    alert = df.tail(TTF)
    #print(df)
    maximum_price = alert['High'].max()   
    minimum_price = alert['Low'].min()
    Wmax = alert['High'].idxmax()
    Wmin = alert['Low'].idxmin() 
    difference = maximum_price - minimum_price   
    #print(difference)
    if Wmax < Wmin:
      LV1 = minimum_price + (difference * 0.236)
    if Wmax > Wmin:
      LV1 = maximum_price - (difference * 0.236)
    #print(LV1)
    #Line(symbol,df,LV1,maximum_price,minimum_price)
    if ( Wmax < Wmin and df['Close'].iloc[-1]>LV1 and df['Close'].iloc[-1]>df['emaposition'].iloc[-1]and df['Close'].iloc[-1]>df['High'].iloc[-2] and df['MACDs'].iloc[-1]>0):
      print(symbol)
      print('Alert')
      notify.send(f'{symbol}\n\nตรวจเช็คเพื่อเข้าทำการ Long') 
      Line(symbol,df,LV1,maximum_price,minimum_price)
    if ( Wmax > Wmin and df['Close'].iloc[-1]<LV1 and df['Close'].iloc[-1]<df['emaposition'].iloc[-1]and df['Close'].iloc[-1]<df['Low'].iloc[-2] and df['MACDs'].iloc[-1]<0):
      print(symbol)
      print('Alert')
      notify.send(f'{symbol}\n\nตรวจเช็คเพื่อเข้าทำการ Short')
      Line(symbol,df,LV1,maximum_price,minimum_price)
  except:
    print(symbol)
    print("data less Than 100" )
  return

#-------------------------------------------------------------------------------
def main():
  seconds = time.time()
  local_time = time.ctime(seconds)
  symbolist = get_symbol()
  for symbol in symbolist:
    df = feed(symbol,TF)
    #print(symbol)
    if mode_trade == 'on':
      trade(symbol,df )     
    if mode_trade == 'off':
      alertonly(symbol,df)
    #time.sleep(1)
  print('Finish')
  return 
#-------------------------------------------------------------------------------
def checkbalance():
  balance = exchange.fetch_balance()
  free_balance = exchange.fetch_free_balance()
  BalanceEntry = free_balance[MarginType]
  notify.send(f'\n สถานะ : SCAN Bot {MarginType}{version} running\nยอดเงินคงเหลือ: {round(BalanceEntry,2)} {MarginType}')
  return
#-------------------------------------------------------------------------------
schedule.every().hour.at(":00").do(checkbalance)

while True:
  seconds = time.time()
  local_time = time.ctime(seconds)
  if alway == 'on':
    main()
  else:
    if TF == '1m' and (str(local_time[16:-5]) == ':00'):
      main()
    if TF == '5m' and (str(local_time[15:-5]) == '5:00' or str(local_time[15:-5]) == '0:00'):
      main()
    if TF == '15m' and (str(local_time[14:-5]) == '00:00' or str(local_time[14:-5]) == '15:00' or str(local_time[14:-5]) == '30:00' or str(local_time[14:-8]) == '45'):
      main()
    if TF == '30m' and (str(local_time[14:-5]) == '00:00' or str(local_time[14:-5]) == '30:00'):
      main()
    if TF == '1h' and str(local_time[14:-5]) == '00:00':
      main()
    if TF == '4h' and str(local_time[11:-5]) == '07:00:00' or str(local_time[11:-5]) == '11:00:00' or str(local_time[11:-5]) == '15:00:00' or str(local_time[11:-5]) == '19:00:00' or str(local_time[11:-5]) == '23:00:00'  :
      main()
    if TF == '1d' and str(local_time[11:-5]) == '07:00:00':
      main()
  if mode_trade == 'on':
    schedule.run_pending()
    
      

Bot start waited time 5m for ScAn
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
LINA/USDT
Alert
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
ATA/USDT
Alert
TRX/USDT
Alert
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
FOOTBALL/USDT
Alert
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
JASMY/USDT
Alert
TRX/USDT
Alert
FOOTBALL/USDT
Alert
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
LINA/USDT
Alert
Finish
Bot check all coin for trade 5m One-way Mode version = 1.0.0...
Finish
Bot check all coin for trade 5m One-way Mode versio

NetworkError: ignored